### Workflow from genomes to families tables
* Setup dir structure
* Cluster ORFs and select reprseqs
* Create HMM profiles with uniclust
* Create db from set of profiles
* Run all vs all comparison of profiles
* Run databases search with profiles
* Collect resulst of all-vs-all into table
* Calculate pairwise coverage of reprseqs

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [ ]:
### setup ###

### imports
# python
import random
import sys
import os
import pandas as pd
import numpy as np
from subprocess import call

# set paths to the virtualenv & repo
# example:
# pipeline_env_path = '/home/user/code/phage-pp-env'
# lib_pp_path       = '/home/user/code/phage-pp-env/phage-pipeline'
pipeline_env_path = '/home/user/code/phage-pp-env' # virtual environment path
lib_pp_path       = '/home/user/code/phage-pp-env/phage-pipeline' # repo path
sys.path.append(lib_pp_path)

# import functions from setup repo
from lib_phage.clustering import cluster_proteins
from lib_phage.utils import setup_dir_tree, fetch_and_rename_protein_ids, build_hhr_table
from lib_phage.utils import process_phanotate_output, create_reprseq_profile_from_clustering
from lib_phage.utils import create_bash_script_to_parse_hhr_results, run_parsing_with_bash
from lib_phage.utils import concatenate_parsing_results, clean_clustering_partial_data
from lib_phage.logs import check_input_repr_prot_selection, validate_output_repr_prot_selection
from lib_phage.logs import check_input_all_vs_all_HMM, save_params_hhblits, validate_output_hhblits
from lib_phage.logs import validate_create_db, validate_search_all_vs_all, validate_input_ECF, validate_output_ECF
from lib_phage.prot_compare import save_individual_seqs, run_hhblits, build_hh_db, run_all_vs_all
from lib_phage.prot_compare import run_hhblits_dbs
from lib_phage.ecf_finder_wrapper import load_and_filter_data, store_scan_results
from lib_phage.repr_hits_pairwise import get_prob_cov

### run mode
# two methods of building profiles of representative sequences were tested:
# hhblits: profiles were build by running hhblits with reprseq as a query on a UniRef database (see paper)
# mmseqs: profiles were build from mmseqs2 results, each cluster converted to MSA
# in the end, the hhblits was selected as final mode of building profiles for reprseq and is recommended mode here
run_mode = 'hhblits' # profile creation mode [mmseqs/hhblits]

### paths
# work dir: select directory where all data will be stored, example:
# work_dir = '/home/user/data/phage-pp-workdir/'
work_dir = '/home/user/data/phage-pp-workdir/'
setup_dir_tree(work_dir)

# binaries and libraries: set paths for external software
mmseqs_binpath  = '/home/user/mmseqs2/bin/mmseqs' # path to mmseqs2/bin/mmseqs
uniref_db_path  = '/mnt/ramdisk/UniRef30_2020_06/UniRef30_2020_06' # path to UniRef database
hhsuite_bins    = '/home/user/hh-suite/bin' # path to hh-suite/bin
hhsuite_scripts = '/home/user/hh-suite/scripts' # path to hh-suite/scripts

### Setup input data:
Copy input set of AA sequences into:<br />
`<work_dir>/input/coding-seqs/`<br />
They should be gathered into single multiple-sequence fasta file.<br />
Name the file `cds-aa.fa` or change filename in the next notebook cell.

### Perform clustering to get representative proteins

In [ ]:
# set clustering params
cluster_params_min_seqid   = 0.3
cluster_params_sensitivity = 7
cluster_params_coverage    = 0.95

# check input files integrity & if this step was already executed:
# if it was warn about data overwrite
if check_input_repr_prot_selection():

    # perform clustering
    clustering_filepath, clustering_msa_filepath = cluster_proteins(input_fasta_filepath=work_dir + 'input/coding-seqs/cds-aa.fa',
                                           output_dirpath=work_dir + 'output/prot-families/representative',
                                           mmseqs_tempdir=work_dir + 'tmp/mmseqs',
                                           mmseqs_binpath=mmseqs_binpath,
                                           cluster_params_min_seqid=cluster_params_min_seqid,
                                           cluster_params_sensitivity=cluster_params_sensitivity,
                                           cluster_params_coverage=cluster_params_coverage,
                                           verbose=True)

    no_repr_prot, name_table_filepath = fetch_and_rename_protein_ids(work_dir, clustering_filepath, 
                                                                     work_dir + 'input/coding-seqs/cds-aa.fa')

    # verify output and save log file
    validate_output_repr_prot_selection(work_dir=work_dir,
                                        output_dirpath=work_dir + 'output/prot-families/representative',
                                        cluster_params_min_seqid=cluster_params_min_seqid,
                                        cluster_params_sensitivity=cluster_params_sensitivity,
                                        cluster_params_coverage=cluster_params_coverage)

### Perform all vs all comparison

In [ ]:
### Create profiles from mmseqs clusters
# used only for mmseqs-based profile creation
# profile for singleton in clustering is the sequence itself

if run_mode == 'mmseqs':
    create_reprseq_profile_from_clustering(clustering_filepath, clustering_msa_filepath,
                                           cds_all_filepath = work_dir + 'input/coding-seqs/cds-aa.fa',
                                           profile_outdir = work_dir + 'intermediate/prot-families/profiles/mmseqs')
else:
    print('Running in hhblits mode, step omitted.')

In [ ]:
### Create profiles for each protein with hhblits

# set create profiles params
cpu  = 16 # max number of CPUs to be used in the step
n    = 1
mact = 0.35
p    = 90
qid  = 10
cov  = 30

if run_mode == 'hhblits': # execute this only when creating profiles with hhblits

    # validate previous step
    if check_input_all_vs_all_HMM(work_dir=work_dir, force=True):

        # execute current step
        save_individual_seqs(work_dir=work_dir)

        run_hhblits(work_dir=work_dir, hhsuite_bins=hhsuite_bins, hhsuite_scripts=hhsuite_scripts, cpu=cpu, 
                    uniref_db_path=uniref_db_path, n=n, mact=mact, p=p, qid=qid, cov=cov)

        # save params to log
        save_params_hhblits(work_dir=work_dir, n=n, mact=mact, p=p, qid=qid, cov=cov)

In [ ]:
### Build database of profiles to be searched with hhsuite
# You can do it two ways:
# 1) Get set of commands that can be run in terminal (print_cmds_only=True)
# 2) Run automatically (print_cmds_only=False)
# First option is recommended as with large amount of data it might be useful to divide steps into smaller chunks.
# Moreover, there was a problem with running cmds in automated way on Linux (for MacOS seemed fine).

if run_mode == 'hhblits': # execute this only when creating profiles with hhblits

    # validate previous step
    if validate_output_hhblits(work_dir, run_mode=run_mode):

        # execute current step
        build_hh_db(work_dir=work_dir, hhsuite_bins=hhsuite_bins,
                    hhsuite_scripts=hhsuite_scripts, verbose=True, run_mode=run_mode, print_cmds_only=True)
        
elif run_mode == 'mmseqs':
    # execute current step
    build_hh_db(work_dir=work_dir, hhsuite_bins=hhsuite_bins,
                hhsuite_scripts=hhsuite_scripts, verbose=True, run_mode=run_mode, print_cmds_only=True)


In [ ]:
### Search all vs all of reprseq profiles

# set all vs all search params
# IMPORTANT: each task takes 2 cpus fo if you have e.g. 32cpus available, set the number below to 16
cpu  = 16 
n    = 1
p    = 50

run_all_vs_all(work_dir=work_dir, hhsuite_bins=hhsuite_bins, 
               hhsuite_scripts=hhsuite_scripts, cpu=cpu, n=n, 
               p=p, a3m_wildcard='reprseq*a3m', run_mode=run_mode)

In [ ]:
### Create results table

# check if previous step complete
if validate_search_all_vs_all(work_dir, run_mode=run_mode):
    
    # create results table
    build_hhr_table(work_dir, run_mode=run_mode)

### Search other databases with reprseqs (e.g. Pfam, ECOD)

In [ ]:
### Perform database search with reprseq profiles

# set db path to search (hh-suite format)
db_path  = '/mnt/ramdisk/Pfam-hh_32_0/pfam'
db_name = 'pfam'

# set database search params
# IMPORTANT: each task takes 2 cpus fo if you have e.g. 32cpus available, set the number below to 16
cpu  = 16
n    = 1
mact = 0.35
p    = 20
qid  = 0
cov  = 0

run_hhblits_dbs(work_dir=work_dir, hhsuite_bins=hhsuite_bins, hhsuite_scripts=hhsuite_scripts, cpu=cpu, 
            db_path=db_path, db_name=db_name, run_mode=run_mode, n=n, mact=mact, p=p, qid=qid, cov=cov)

In [ ]:
### Gather results from db search into table - parallel
# parsing of result hhr files is done in parallel to speed it up

# create script for parallel parsing
create_bash_script_to_parse_hhr_results(work_dir, pipeline_env_path, lib_pp_path)

In [ ]:
# run parsing script
n_cores = 32
run_parsing_with_bash(work_dir, n_cores, db_name)

In [ ]:
# check if all files were parsed and gather results into one file
if concatenate_parsing_results(work_dir, db_name):
    clean_clustering_partial_data(work_dir, db_name)

### Run pairwise comparison of the coverage of reprseqs

In [ ]:
### Define paths
output_dir = work_dir + 'output/'
inter_dir = work_dir + 'intermediate/'
all_by_all_output_dir = output_dir + 'prot-families/all-by-all/' + run_mode + '/'
families_output_dir = inter_dir + 'prot-families/families/'
reprseq_output_dir = output_dir + 'prot-families/representative/'
parts_dir = work_dir + 'tmp/prot-families/pair_table_chunks/'

In [ ]:
### Load & filter table
# load hhr_table
table_hhr_filename = all_by_all_output_dir + 'table-hhr.txt'
table_hhr = pd.read_csv(table_hhr_filename, sep=',')
table_hhr

In [ ]:
# filter table by probability value
prob_threshold = 50
table_hhr = table_hhr[table_hhr['prob'] >= prob_threshold]
table_hhr

In [ ]:
# get unique pairs
# get from table_hhr qname-sname unique pairs of ids
# drop duplicated pairs (i.e. keep only one from A->B and B->A)

pair_table = table_hhr[['qname', 'sname']]
pair_table = pair_table[pair_table['qname'] != pair_table['sname']]
pair_table = pair_table.drop_duplicates()

pair_table['names'] = pair_table.apply(lambda row: str(sorted([row.qname, row.sname])), axis=1)
pair_table.drop_duplicates(subset=['names'], inplace=True)
pair_table.drop(columns=['names'], inplace=True)

pair_table

In [ ]:
### Run comparison
# create script to run in parallel
# dataset will also be divided into smaller chunks to optimize computation
n_cores = 32
n_subsets = 10

In [ ]:
# divide pair table into sub-tables
chunk_size = int(len(pair_table) / n_cores)
pair_table_parts = []
for i in range(n_cores-1):
    pair_table_parts.append(pair_table[i*chunk_size:(i+1)*chunk_size])
pair_table_parts.append(pair_table[(n_cores-1)*chunk_size:])

for i, pair_table_part in enumerate(pair_table_parts):
    pair_table_part.to_csv(parts_dir + 'pair-table-' + str(i) + '.csv', index=False)

In [ ]:
# now divide into sub-subtables for smaller mem usage
for i in range(n_cores):
    pair_table_part = pd.read_csv(parts_dir + 'pair-table-' + str(i) + '.csv')
    pair_table_subsets = np.array_split(pair_table_part, n_subsets)

    for j, pair_sub in enumerate(pair_table_subsets):
        pair_sub.to_csv(parts_dir + 'pair-table-' + str(i) + '-' + str(j) + '.csv', index=False)
    os.remove(parts_dir + 'pair-table-' + str(i) + '.csv')

In [ ]:
# create subtables of results to optimize mem usages
for i in range(n_cores):
    for j in range(n_subsets):
        pair_sub = pd.read_csv(parts_dir + 'pair-table-' + str(i) + '-' + str(j) + '.csv')
        qname_list = list(pair_sub['qname'].unique())
        sname_list = list(pair_sub['sname'].unique())
        
        df1 = table_hhr[(table_hhr['qname'].isin(qname_list)) & (table_hhr['sname'].isin(sname_list))]
        df2 = table_hhr[(table_hhr['qname'].isin(sname_list)) & (table_hhr['sname'].isin(qname_list))]
        df = pd.concat([df1, df2])
        df.drop_duplicates(inplace=True)
        
        df.to_csv(parts_dir + 'table-hhr-' + str(i) + '-' + str(j) + '.csv', index=False)

In [ ]:
# parallel python script to process sub-table
run_cores = [0,32] # subset to run

script_filepath =  work_dir + 'tmp/prot-families/run-pairwise-hits.sh'
ecf=False

cmd = '#!/bin/bash\n\n'
cmd += 'source ' + pipeline_env_path + '/bin/activate\n'

for i in range(run_cores[0], run_cores[1]):
    pair_table_path = parts_dir + 'pair-table-' + str(i) + '.csv'    
    cmd += 'nohup python3 {}/lib_phage/run_hits_pairwise_single_table.py {} {} {} {} {} {} {} &\n'.format(
    pipeline_env_path, work_dir, run_mode, i, prob_threshold, n_subsets, ecf, lib_pp_path)

with open(script_filepath, 'w') as file_sh:
    file_sh.write(cmd)
    
print('RUN MANUALLY script from ' + work_dir + 'tmp/prot-families')

#### Make sure you run script as instructed by previous cell

In [ ]:
# concatenate results, add header
part_results_filepath = families_output_dir + 'repr-hits-pairwise-prob' + str(prob_threshold) + '-*.txt'
concat_results_filepath = output_dir + 'prot-families/families/repr-hits-pairwise-prob' + str(prob_threshold) + '.txt'
header_path = output_dir + 'prot-families/families/header'

call('echo "qname,sname,prob,scov_min,scov_max,qcov_min,qcov_max,pident,max_cov,min_cov" > ' + header_path, shell=True)
call('cat ' + part_results_filepath + ' > ' + concat_results_filepath, shell=True)
call('cat ' + header_path + ' ' + concat_results_filepath + ' > ' + concat_results_filepath.replace('.txt', '.csv'),
     shell=True)

### Prepare file for MCL clustering of reprseqs by pairwise comparison (group into protein families)

In [ ]:
### Create MCL input
pair_hits_filename = 'repr-hits-pairwise-prob' + str(prob_threshold) + '.csv'
pair_hits_filepath = output_dir + 'prot-families/families/' + pair_hits_filename

pair_table_full = pd.read_csv(pair_hits_filepath)
pair_table_full['qcov'] = pair_table_full.apply(lambda x: max(x.qcov_min, x.qcov_max), axis=1)
pair_table_full['scov'] = pair_table_full.apply(lambda x: max(x.scov_min, x.scov_max), axis=1)
pair_table_full['fam_cov'] = pair_table_full.apply(lambda x: min(x.qcov, x.scov), axis=1)
pair_table_full['weight'] = pair_table_full.apply(lambda x: x.prob * min(x.qcov, x.scov), axis=1)

pair_table_full

In [ ]:
# filter table by qcov & scov values
family_cov = 0.8
family_prob = 0.95
pair_table_full = pair_table_full[pair_table_full['prob'] >= family_prob]
pair_table_full = pair_table_full[pair_table_full['fam_cov'] >= family_cov]
pair_table_full

In [ ]:
# create MCL input table
pair_table_mcl = pair_table_full[['qname', 'sname', 'weight']]
pair_table_mcl

In [ ]:
# add singletons, i.e. all reprseq that does not have any hit and therefore are out of the table
# encode singletons as: reprseq_id, reprseq_id, 1
reprseq_nametable = pd.read_csv(reprseq_output_dir + 'name-table.txt')
qset = set(pair_table_mcl['qname'].unique())
sset = set(pair_table_mcl['sname'].unique())
reprseq_present = qset.union(sset)

rseq_names = set(reprseq_nametable['repr.name'].unique())
rseq_missing = rseq_names.difference(reprseq_present)
len(rseq_missing)

rseq_singletons = {i:[rseq_name, rseq_name, 1.0] for i, rseq_name in enumerate(list(rseq_missing))}
rseq_singletons_table = pd.DataFrame.from_dict(rseq_singletons, orient='index', columns=['qname', 'sname', 'weight'])
rseq_singletons_table

In [ ]:
# concatenate filtered table with singleton table
pair_table_mcl_all = pd.concat([pair_table_mcl, rseq_singletons_table])
pair_table_mcl_all

In [ ]:
# drop final input file for MCL
# perform MCL outside of this notebook
pair_table_mcl_all.to_csv(pair_hits_filepath.replace('.csv', '-cov' + str(int(family_cov*100)) + '-mcl-in.abc'), sep=' ', index=False, header=False, float_format="%.4f")